In [372]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [373]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [374]:
data_space_provider = "http://0.0.0.0:1200"
data_space_consumer = "http://0.0.0.0:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding

### Current Participants information

In [375]:
url = data_space_provider + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "urn:uuid:211598c9-b045-41fd-8f6e-7e71ade0718d",
    "participant_slug": "Provider",
    "participant_type": "Provider",
    "base_url": "http://0.0.0.0:1200",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-06-18T23:45:20.603413",
    "last_interaction": "2025-06-18T23:45:20.603807",
    "is_me": true
}


In [376]:
url = data_space_consumer + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "urn:uuid:33b0987a-6f04-488a-b28b-ed092888e953",
    "participant_slug": "Consumer",
    "participant_type": "Consumer",
    "base_url": "http://0.0.0.0:1100",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-06-18T23:45:18.887339",
    "last_interaction": "2025-06-18T23:45:18.887441",
    "is_me": true
}


### Consumer registering in Provider via Verifiable presentation

In [381]:
url = data_space_consumer + "/auth/manual/ssi"
payload = {
    #"url": data_space_provider + "/access",
    "url":  "http://host.docker.internal:1200/access",
    "id": "friendly_slugname_for_provider",
    "actions": "talk"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2F0.0.0.0%3A1200%2Fverify%2FhQp6YpP1nCd8&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2F0.0.0.0%3A1200%2Fpd%2FhQp6YpP1nCd8&client_id_scheme=redirect_uri&nonce=GLWzPXaf3RsW&response_uri=http%3A%2F%2F0.0.0.0%3A1200%2Fverify%2FhQp6YpP1nCd8'

### Provider and Consumer ids

In [380]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json[1]["participant_id"]
data_space_consumer_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

IndexError: list index out of range

In [317]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json[1]["participant_id"]
data_space_provider_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "urn:uuid:4f35a3e1-9ad8-4238-94ca-96d97ca207a9",
        "participant_slug": "Consumer",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": null,
        "token_actions": null,
        "saved_at": "2025-06-17T11:34:25.652453",
        "last_interaction": "2025-06-17T11:34:25.652456",
        "is_me": true
    },
    {
        "participant_id": "urn:uuid:ead1a1f7-340b-43b0-b909-ec420b3d666e",
        "participant_slug": "friendly_slugname_for_provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": "cIGWuxexU6tC-1A8PdOtR75_UcDpzPOk9smbezPVKfk",
        "token_actions": "talk",
        "saved_at": "2025-06-17T11:35:00.445939",
        "last_interaction": "2025-06-17T11:35:00.445940",
        "is_me": false
    }
]


## Create Catalog instances and metadata

In [318]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:4ec234d9-7c5b-4eb5-846b-f491dda5725d'

In [319]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce'

In [320]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:4ad62351-9cfc-4841-8d3f-8c7dcacf77ca'

In [321]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

'urn:uuid:ea660dc9-17d3-464b-ad96-ad40c37565c9'

## Create policies

In [322]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:8c89eac6-e253-4719-87aa-8266c115d48d",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce"
}


## Contract Negotiation

In [323]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "offer": {
    "@id": "urn:uuid:8c89eac6-e253-4719-87aa-8266c115d48d",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "REQUESTED"
  }
}


In [324]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:6a1f9cab-f594-4f42-8891-ccf611c74513",
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "offer": {
    "@id": "urn:uuid:8c89eac6-e253-4719-87aa-8266c115d48d",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "OFFERED"
  }
}


In [325]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "offer": {
    "@id": "urn:uuid:8c89eac6-e253-4719-87aa-8266c115d48d",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "REQUESTED"
  }
}


In [326]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:6a1f9cab-f594-4f42-8891-ccf611c74513",
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "offer": {
    "@id": "urn:uuid:8c89eac6-e253-4719-87aa-8266c115d48d",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "OFFERED"
  }
}


In [327]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "ACCEPTED"
  }
}


In [328]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "agreement": {
    "@id": "urn:uuid:18ae965a-1931-41da-9732-dee1bcde450a",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:uuid:6b76f826-7804-4dbf-82f4-25a8d926a3ce",
    "assigner": "urn:uuid:2436e82a-f658-49f0-a979-684e4c054f6d",
    "assignee": "urn:uuid:6a1f9cab-f594-4f42-8891-ccf611c74513",
    "timestamp": "2025-06-18T22:35:20.063015+00:00"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "AGREED"
  }
}


In [329]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "VERIFIED"
  }
}


In [330]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
  "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:2d7d1544-1bab-4cf2-a6af-ad93a1ddd0dc",
    "consumerPid": "urn:uuid:84d131f1-271d-445d-ae25-9c93fa572a56",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [331]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
  "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "agreementId": "urn:uuid:18ae965a-1931-41da-9732-dee1bcde450a",
  "callbackAddress": "http://127.0.0.1:1100/urn:uuid:8c80609f-d0e6-48c7-84ca-f9afd91e0057",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "state": "REQUESTED"
  }
}


In [332]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
  "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://127.0.0.1:1200/data/urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "state": "STARTED"
  }
}


In [333]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://127.0.0.1:1100/data/urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-06-18T22:35:20.829505",
  "updated_at": "2025-06-18T22:35:20.894450",
  "state": "STARTED"
}


In [334]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)


     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [335]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
  "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "state": "SUSPENDED"
  }
}


In [336]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

HTTP Error 403: Forbidden


In [337]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
  "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "state": "STARTED"
  }
}


In [338]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [339]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
  "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "state": "SUSPENDED"
  }
}


In [340]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
  "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:590cb615-954b-4e2a-967c-ed8f162d60f1",
    "consumerPid": "urn:uuid:b3324461-ae16-42b8-a3e5-cdeeedb4528f",
    "state": "COMPLETED"
  }
}
